# Pandas internationalisation: Bangala (বাংলা) data example

An example of reading in Bangala data in Pandas.

The file `/data/bn_global_popl.tsv` is a tab delimited file in Bangla (Bengali). 

Column 0 (নং) contains a ranking expressed in Bangla digits. Column 3 (জাতিসংঘের অনুমান) contains a integer in Bangla digits. Column 4 (তারিখ) contains dates that are formatted according to Bangla locale preferences.

A set of conversion functions are used with `pd.read_table()` to convert the data to a format that can be used in Pandas.

Columns 0 and 3 are converted from Bangla digits to Arabic digits, using the `convert_digits()` function. Column 4 is converted to the ISO 8601 date format, using `icu.DateFormat` and `icu.SimpleDateFormat`.

`icu.DateFormat` &ndash; a class for parsing (reading) and formating (writing) dates for any defined locale. \
`icu.SimpleDateFormat` &ndash; a class for language-independant parsing (reading) and formating (writing) dates. It is a concrete subclass of `icu.DateFormat`.

In [4]:
import unicodedataplus as ud, regex as re, pandas as pd
from icu import Locale, DateFormat, SimpleDateFormat

def convert_digits(s, sep = (",", ".")):
    nd = re.compile(r'^-?\p{Nd}[,.\u066B\u066C\u0020\u2009\p{Nd}]+$')
    tsep, dsep = sep
    if nd.match(s):
        s = s.replace(tsep, "")
        s = ''.join([str(ud.decimal(c, c)) for c in s])
        if dsep in s:
            return float(s.replace(dsep, ".")) if dsep != "." else float(s)
        return int(s)
    return s

loc = Locale("bn_IN")
dformat = DateFormat.LONG
inf = DateFormat.createDateInstance(dformat, loc)
outf = SimpleDateFormat("yyyy-MM-dd")

def convert_dates(ld, inf, outf):
    in_date = inf.parseObject(ld)
    return outf.format(in_date)

lf = lambda x: convert_dates(x, inf=inf, outf=outf)

In [7]:
conv = {'জাতিসংঘের অনুমান': convert_digits, 'নং': convert_digits, 'তারিখ': lf}
df = pd.read_table("../data/bn_global_popl.tsv", skiprows=range(1, 3), converters=conv, parse_dates=['তারিখ'])
df.head(3)

,নং,দেশ,মহাদেশ,জাতিসংঘের অনুমান,তারিখ
0,1,গণচীন,এশিয়া,1433783686,2019-07-01
1,2,ভারত,এশিয়া,1366417754,2019-07-01
2,3,যুক্তরাষ্ট্র,আমেরিকা,329064917,2019-07-01


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233 entries, 0 to 232
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   নং                233 non-null    int64         
 1   দেশ               233 non-null    object        
 2   মহাদেশ            233 non-null    object        
 3   জাতিসংঘের অনুমান  233 non-null    int64         
 4   তারিখ             233 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 9.2+ KB


## Resources

* [Formatting Dates and Times](https://unicode-org.github.io/icu/userguide/format_parse/datetime/)
* [icu::DateFormat Class Reference](https://unicode-org.github.io/icu-docs/apidoc/dev/icu4c/classicu_1_1DateFormat.html)
* [icu::SimpleDateFormat Class Reference](https://unicode-org.github.io/icu-docs/apidoc/released/icu4c/classSimpleDateFormat.html)